In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.autograd.functional import hessian, jacobian

In [111]:
x = np.linspace(0,1,10, dtype=np.float)
t = np.linspace(0,1,10, dtype=np.float)
x, t = np.meshgrid(x, t) 
x = x.flatten()
t = t.flatten()
#X = np.concatenate((x.reshape(-1,1), t.reshape(-1, 1)), axis=1)

num_points, input_dim = X.shape
hidden_neurons = 100
output_dim = 1

In [124]:
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

xt = tf.convert_to_tensor(X, dtype="float32")

class ODE_FFNN(tf.keras.Model):
    def __init__(self):
        super(ODE_FFNN, self).__init__()
        self.input_layer = tf.keras.layers.Dense(2)
        self.hidden_layer = tf.keras.layers.Dense(10, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1)
        return
        
    def call(self, input_tensor, training=True):
        x = self.input_layer(input_tensor, training=training)
        x = self.hidden_layer(x, training=training)
        x = self.output_layer(x, training=training)
        return x
    
def g_trial(xt, N):
    x, t, = xt[:, 0], xt[:, 1]
    x = tf.reshape(x, shape=(-1,1))
    t = tf.reshape(t, shape=(-1,1))
    #print(N(xt).shape)
    #print(x.shape)
    #print(t.shape)
    return (1 - t) * tf.sin(np.pi * t) + x * (1 - x) * t * N(xt)


    
def cost_func(x, nn):
    return tf.reduce_mean(x, axis=0)
    
nn = ODE_FFNN()

#print(nn(xt).shape)
print("->", g_trial(xt, nn).shape)

print(tf.hessians(ys = g_trial(xt, nn), xs = xt))

#cost_func(xt, nn)

-> (100, 1)
[<tf.Tensor 'Reshape_177:0' shape=(100, 2, 100, 2) dtype=float32>]


In [128]:
def g_bc(x, t):
    return (1 - t) * torch.sin(np.pi * x)

def g_nn(x, t, N):
    return x * (1 - x) * t * N

def g_trial(val, N):
    x, t = val.T
    return g_bc(x, t) + g_nn(x, t, N)

def ode_loss(x, N):
    #dudx2 = hessian(g_trial, (x, N))
    #dudt = jacobian(g_trial, (x, N))
    
    loss_all = torch.zeros(x.size(0), requires_grad = True)
    
    # Split into each x_i, N_i pair before computing gradient!
    for i in range(x.size(0)):
        #dudx2 = hessian(g_trial, (x[i,:], N[i]))[0][0][0][0]
        J_g, _ = jacobian(g_trial, (x[i,:], N[i]))
        H_g, _ = hessian(g_trial, (x[i,:], N[i]))
        
        #print(J_g[0,1])
        #print(H_g[0][0,0])
        
        #print("\n\ndudx = ", dudx2,"\ndudt=", dudt)
        loss_all[i] = (J_g[0,1] - H_g[0][0,0])**2
        
    #print("loss=",loss)
    #return torch.sum(g_trial(x, N))
    return torch.sum(loss_all) / x.size(0)
# https://pytorch.org/docs/stable/autograd.html#torch.autograd.functional.hessian
#u_hess = hessian(g_trial, (x, N))
# https://pytorch.org/docs/stable/autograd.html#torch.autograd.functional.hessian
#u_jacob = jacobian(g_trial, (x, N))

#torch.autograd.set_detect_anomaly(True)


dtype = torch.float
#device = torch.device("cuda:0")
device = torch.device("cpu")

x = torch.from_numpy(X)
x = x.to(dtype)
x.requires_grad = True

# Create random Tensors for weights.
w1 = torch.randn(input_dim, hidden_neurons, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_neurons, output_dim, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-7
for t in range(5):
    N_output = F.relu(x.mm(w1)).mm(w2)
    
    # Compute and print loss
    loss = ode_loss(x, N_output)
    print("loss = ",loss)
    # Use autograd to compute the backward pass.
    #loss.backward()
    # Update weights using gradient descent
    #with torch.no_grad():
    #    w1 -= learning_rate * w1.grad
    #    w2 -= learning_rate * w2.grad
    #    # Manually zero the gradients after updating weights
    #    w1.grad.zero_()
    #    w2.grad.zero_()

loss =  tensor(32.5826, grad_fn=<DivBackward0>)
loss =  tensor(32.5826, grad_fn=<DivBackward0>)
loss =  tensor(32.5826, grad_fn=<DivBackward0>)
loss =  tensor(32.5826, grad_fn=<DivBackward0>)
loss =  tensor(32.5826, grad_fn=<DivBackward0>)
